## Imports

In [1]:
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram

import numpy as np
import math
import random

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import cm

## Dataset generation

Output of the old generator
```python
[(2,
  array([45, 11, 40, 38]),
  {(0, 2): -15, (0, 3): -9, (1, 2): -9, (1, 3): -14}),
 (2,
  array([10, 37,  9, 46]),
  {(0, 2): -20, (0, 3): -15, (1, 2): -2, (1, 3): -4}),
 (2,
  array([12, 48, 35, 38]),
  {(0, 2): -3, (0, 3): -7, (1, 2): -19, (1, 3): -8}),
 (2,
  array([ 4, 42, 28, 33]),
  {(0, 2): -11, (0, 3): -10, (1, 2): -1, (1, 3): -14}),
 (2,
  array([43, 23, 23, 18]),
  {(0, 2): -3, (0, 3): -2, (1, 2): -16, (1, 3): 0})]
  ```

In [2]:
def create_savings(n_queries, n_plans_per_query):
    savings = {}
    for i in range(n_queries-1):
        for j in range(n_plans_per_query[i]):
            s = j + np.sum(n_plans_per_query[0:i], dtype=int)
            for a in range(i+1, n_queries):
                for b in range(n_plans_per_query[a]):
                    t = b + np.sum(n_plans_per_query[:a], dtype=int)
                    savings[s, t] = random.randint(-20, 0)

    return savings

In [3]:
def create_problems(n_problems, n_queries, n_plans_per_query, cost_min = 0, cost_max = 50, savings_min = -20, savings_max = 0):
    problems = []
    for i in range(n_problems):
        problems.append((n_plans_per_query, np.random.randint(cost_min, cost_max, np.sum(n_plans_per_query)), 
            create_savings(n_queries, n_plans_per_query)))
    return problems

Problems are generated, but only work, for now, in double combinations -> no three way savings!

In [71]:
problems = create_problems(1, 3, [2,3,2])
problems

[([2, 3, 2],
  array([22, 40, 16, 19, 26, 15, 35]),
  {(0, 2): -7,
   (0, 3): -7,
   (0, 4): -5,
   (0, 5): -3,
   (0, 6): -9,
   (1, 2): -9,
   (1, 3): -17,
   (1, 4): 0,
   (1, 5): -3,
   (1, 6): -19,
   (2, 5): -4,
   (2, 6): -13,
   (3, 5): -10,
   (3, 6): -7,
   (4, 5): -18,
   (4, 6): -19})]

We now want to classically solve the problem so we can get a ranking.

In [72]:
#def solve_classical(problems):
solutions_complete = []
best_solution = []
end_costs = {}
for problem in problems:
    t_cost_2d = {}
    cost = 0
    ##TO MAKE DYNAMIC YOU HAVE TO ITERATE FOR EACH QUERY
    for x, y in problem[2]:
        t_cost_2d[x, y] = problem[1][x] + problem[1][y] + problem[2][x, y]
    t_cost_3d = {}
    for a, b in problem[2]:
        for _, c in [(x, y) for x, y in problem[2] if x == b]:
            t_cost_3d[a, b, c] = t_cost_2d[a, b] + t_cost_2d[a, c] + t_cost_2d[b, c]

print(problems[0][-1])
print(t_cost_2d)
print(t_cost_3d)
#   return solutions_complete

{(0, 2): -7, (0, 3): -7, (0, 4): -5, (0, 5): -3, (0, 6): -9, (1, 2): -9, (1, 3): -17, (1, 4): 0, (1, 5): -3, (1, 6): -19, (2, 5): -4, (2, 6): -13, (3, 5): -10, (3, 6): -7, (4, 5): -18, (4, 6): -19}
{(0, 2): 31, (0, 3): 34, (0, 4): 43, (0, 5): 34, (0, 6): 48, (1, 2): 47, (1, 3): 42, (1, 4): 66, (1, 5): 52, (1, 6): 56, (2, 5): 27, (2, 6): 38, (3, 5): 24, (3, 6): 47, (4, 5): 23, (4, 6): 42}
{(0, 2, 5): 92, (0, 2, 6): 117, (0, 3, 5): 92, (0, 3, 6): 129, (0, 4, 5): 100, (0, 4, 6): 133, (1, 2, 5): 126, (1, 2, 6): 141, (1, 3, 5): 118, (1, 3, 6): 145, (1, 4, 5): 141, (1, 4, 6): 164}


In [74]:
#def solve_classical(problems):
costs = problem[2]
for problem in problems:
    while len(costs) > np.prod(problem[0]):
        new_costs = {}
        for a in costs:
            for b in [z for z in costs if z[0] == a[-1]]:
                c = list(a)
                c.append(b[-1])
                c = tuple(c)
                new_costs[c] = costs[a] + costs[b]
            print(new_costs)
        if new_costs == costs:
            break
        else:
            costs = new_costs

print(costs)
#   return solutions_complete

{(0, 2, 5): -11, (0, 2, 6): -20}
{(0, 2, 5): -11, (0, 2, 6): -20, (0, 3, 5): -17, (0, 3, 6): -14}
{(0, 2, 5): -11, (0, 2, 6): -20, (0, 3, 5): -17, (0, 3, 6): -14, (0, 4, 5): -23, (0, 4, 6): -24}
{(0, 2, 5): -11, (0, 2, 6): -20, (0, 3, 5): -17, (0, 3, 6): -14, (0, 4, 5): -23, (0, 4, 6): -24}
{(0, 2, 5): -11, (0, 2, 6): -20, (0, 3, 5): -17, (0, 3, 6): -14, (0, 4, 5): -23, (0, 4, 6): -24}
{(0, 2, 5): -11, (0, 2, 6): -20, (0, 3, 5): -17, (0, 3, 6): -14, (0, 4, 5): -23, (0, 4, 6): -24, (1, 2, 5): -13, (1, 2, 6): -22}
{(0, 2, 5): -11, (0, 2, 6): -20, (0, 3, 5): -17, (0, 3, 6): -14, (0, 4, 5): -23, (0, 4, 6): -24, (1, 2, 5): -13, (1, 2, 6): -22, (1, 3, 5): -27, (1, 3, 6): -24}
{(0, 2, 5): -11, (0, 2, 6): -20, (0, 3, 5): -17, (0, 3, 6): -14, (0, 4, 5): -23, (0, 4, 6): -24, (1, 2, 5): -13, (1, 2, 6): -22, (1, 3, 5): -27, (1, 3, 6): -24, (1, 4, 5): -18, (1, 4, 6): -19}
{(0, 2, 5): -11, (0, 2, 6): -20, (0, 3, 5): -17, (0, 3, 6): -14, (0, 4, 5): -23, (0, 4, 6): -24, (1, 2, 5): -13, (1, 2, 6): -22,

In [75]:
for a in costs:
    for b in a:
        costs[a] += problems[0][1][b]

In [86]:
bit_strings = []
for a in costs:
    b = list('0'*sum(problems[0][0]))
    for i in a:
        b[i] = '1'
    bit_strings.append(''.join(b))
bit_strings

['1010010',
 '1010001',
 '1001010',
 '1001001',
 '1000110',
 '1000101',
 '0110010',
 '0110001',
 '0101010',
 '0101001',
 '0100110',
 '0100101']

We now generate the combinational bitstrings that are possible

In [ ]:
n_qubits = np.sum(problems[0][0])
binary_string = []
for i, v in enumerate(problems[0][0]):
    if i == 0:
        for j in range(v):
            binary_string.append('0'*j + '1' + '0'*(v-j-1))
    else:
        copy = []
        for x in binary_string:
            for j in range(v):
                copy.append(x + '0'*j + '1' + '0'*(v-j-1))
        binary_string = copy


print(binary_string)


['1010100', '1010010', '1010001', '1001100', '1001010', '1001001', '0110100', '0110010', '0110001', '0101100', '0101010', '0101001']


Now we generate the circuits...

In [ ]:
from qiskit import *
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

import numpy as np
import math

In [ ]:
problems = create_problems(1, 3, [2,3,2])
problems

[([2, 3, 2],
  array([23, 47, 11, 25, 47, 43, 42]),
  {(0, 2): -19,
   (0, 3): 0,
   (0, 4): -20,
   (0, 5): -11,
   (0, 6): -8,
   (0, 7): -5,
   (1, 2): -4,
   (1, 3): -13,
   (1, 4): 0,
   (1, 5): -2,
   (1, 6): -10,
   (1, 7): -19,
   (2, 5): -6,
   (2, 6): -6,
   (3, 5): -19,
   (3, 6): -8,
   (4, 5): -14,
   (4, 6): -12})]

In [ ]:
circuit = QuantumCircuit(np.sum(problems[0][0]))
circuit.h(range(circuit.width()))
for i, v in enumerate(problems[0][1]):
    circuit.ry(v, i)
circuit.barrier()

prev_i = 0
for i, v in problems[0][2]:
    if prev_i != i:
        circuit.barrier()
    circuit.crz(problems[0][2][i, v], i, v)
    prev_i = i
circuit.barrier()

circuit.draw()

CircuitError: 'Index out of range.'

In [ ]:
np.random.randint(1,4, size=5)

array([1, 3, 2, 2, 3])